In [ ]:
import glob
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.applications.vgg16 import VGG16
#from cv2 import imshow

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers.legacy import Adam

2023-06-07 22:45:46.490194: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 22:45:46.931578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-07 22:45:46.931634: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-07 22:45:48.387272: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Create model

In [ ]:
img_height = 256
img_width = 256

In [ ]:
def create_model(input_shape, optimizer='rmsprop', fine_tune=0):
  conv_base = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  if fine_tune > 0:
      for layer in conv_base.layers[:-fine_tune]:
          layer.trainable = False
  else:
      for layer in conv_base.layers:
          layer.trainable = False

  top_model = conv_base.output

  top_model = Flatten(name="flatten")(top_model)
  top_model = Dense(4096, activation='relu')(top_model)
  top_model = Dense(1072, activation='relu')(top_model)
  top_model = Dropout(0.2)(top_model)
  output_layer = Dense(1, activation='sigmoid')(top_model)
  
  model = Model(inputs=conv_base.input, outputs=output_layer)

  model.compile(optimizer=optimizer, 
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  return model


In [ ]:
#train_ds = train_ds.take(330)
#test_ds = test_ds.take(330)


# Set up N parties

In [ ]:
# Set number of agents
numOfParties = 5

# number of dataset examples each agent
local_num_examples = [1000, 1000, 1000, 1000, 1000]
totNumOfExamples = sum(local_num_examples)

# Model weights aggregation

In [ ]:
input_shape = (img_height, img_width, 3)
optim_1 = Adam(learning_rate=0.001)
n_epochs = 1
batch_size = 32

### Load encrypted model weights

In [ ]:
list_of_pickle_files = list(glob.glob('./received_pickles/*'))

In [ ]:
agents_encrypted_weights = []
for file_path in list_of_pickle_files:
  with open(file_path, 'rb') as file:
    loaded_weights = pickle.load(file)
    agents_encrypted_weights.append(loaded_weights)

In [ ]:
agents_decrypted_weights = []
# Decrypt agents weights
for agent_encrypted_weights in agents_encrypted_weights:
  agents_decrypted_weights.append(HE_decrypt(agent_encrypted_weights))

### Aggregate model weights

In [ ]:
global_model = create_model(input_shape, optim_1, fine_tune=0)

In [ ]:
# update global model with mean of weights
avg_weights = []
for j in range(len(agents_decrypted_weights[0])):  # for each layer in the model
    layer_weights = [agents_decrypted_weights[i][j] for i in range(len(agents_decrypted_weights))]  # collect this layer's weights from each client
    layer_weights_avg = np.average(layer_weights, axis=0, weights=[num / totNumOfExamples for num in local_num_examples])  # compute the weighted average
    avg_weights.append(layer_weights_avg)  # append the average weights for this layer

In [ ]:
global_model.set_weights(avg_weights)

In [ ]:
global_model.save("model.h5")

### Encrypt model weights and send them to every agents

In [ ]:
global_weights = global_model.get_weights()
encrypted_global_weights = HE_encrypt(global_weights)
# Now send the aggregated model weights to every agents